In [ ]:
import os
import numpy as np
from PIL import Image
import matplotlib
from matplotlib import pyplot,pylab
plt = pyplot
import scipy
import seaborn as sns
sns.set_style('white')
import string
import pandas as pd
import json
import pymongo as pm
from glob import glob
from IPython.display import clear_output
import itertools 
from random import sample
import importlib
from PIL import Image

### set up project paths

In [ ]:
## root paths
curr_dir = os.getcwd()
proj_dir = os.path.abspath(os.path.join(curr_dir,'../..'))
data_dir = os.path.abspath(os.path.join(curr_dir,'../..','data')) ## use relative paths
stim_dir = os.path.abspath(os.path.join(proj_dir,'stimuli'))
analysis_dir = os.path.abspath(os.path.join(curr_dir,'../..','analysis')) ## use relative paths
plot_dir =  os.path.abspath(os.path.join(curr_dir,'../..','results','plots'))
csv_dir = os.path.join(proj_dir, 'results','csv')
sketch_dir = os.path.abspath(os.path.join(proj_dir,'sketches'))
catch_dir = os.path.abspath(os.path.join(proj_dir,'stimuli/classify_iternum/catch_trial_stimuli'))    # we want to get the stims for the catch trials in our rating task


## add helpers to python path
import sys
if os.path.join(proj_dir, 'utils') not in sys.path:
    sys.path.append(os.path.join(proj_dir, 'utils'))
import utils as h    


### inspect stims and get image paths 

In [ ]:
## get sketch paths
path_to_sketches = sketch_dir
im_list = h.list_files(sketch_dir)
assert len(im_list)==32*61 # num trials per game * num kosher games

## get catch stim paths
path_to_catches = catch_dir
catch_list = h.list_files(catch_dir)
assert len(catch_list)==12 # 8 number catch stims and 4 shape ones


In [ ]:
## sort stims in place
importlib.reload(h)
h.sort_nicely(im_list)


In [ ]:
## manually inspect the images
imsize = 256
fname = im_list[500]
print(fname)
Image.open(fname).resize((imsize,imsize))

### upload stims to s3

In [ ]:
import boto3
import botocore

In [ ]:
bucket_name = 'iternum-sketches'
full_sketch_paths = h.list_files(path_to_sketches)
h.sort_nicely(full_sketch_paths)

In [ ]:
## helper to speed things up by not uploading images if they already exist, can be overriden 
def check_exists(s3, bucket_name, stim_name):
    try:
        s3.Object(bucket_name,stim_name).load()    
        return True
    except botocore.exceptions.ClientError as e:    
        if (e.response['Error']['Code'] == "404"):
            print('The object does not exist.')
            return False
        else:
            print('Something else has gone wrong with {}'.format(stim_name))
            


In [ ]:
print(check_exists(boto3.resource('s3'), bucket_name, fname.split('/')[-1]))
print(fname.split('/')[-1])
# print(len(im_list))

# print(full_sketch_paths[5])

In [ ]:

# access the password and username for s3 from our local directory
access_id = pd.read_csv(os.path.abspath(os.path.join(curr_dir,'../../../.aws/credentials')), header = None).values[1][0].split('=')[-1]          
access_key = pd.read_csv(os.path.abspath(os.path.join(curr_dir,'../../../.aws/credentials')), header = None).values[2][0].split('=')[-1]      


In [ ]:
reallyRun = 0
if reallyRun: 
    ## tell user some useful information
    print('Path to stimuli is : {}'.format(path_to_sketches))
    print('Uploading to this bucket: {}'.format(bucket_name))

    ## establish connection to s3 
#     s3 = boto3.resource('s3')
    s3 = boto3.resource('s3', 
                  region_name='us-east-1', 
                  aws_access_key_id=access_id, #"STORE THIS LOCALLY! Boto3 should be accessing ~/.aws/credentials but it's not for some reason",     
                  aws_secret_access_key= access_key ) #'THIS TOO!'
    
    ## create a bucket with the appropriate bucket name
    try: 
        b = s3.create_bucket(Bucket=bucket_name) 
        print('Created new bucket.')
    except Exception as e:
        print(e)
        b = s3.Bucket(bucket_name)
        print('Bucket already exists.')

    ## do we want to overwrite files on s3?
    overwrite = False
    
    ## set bucket and objects to public
    b.Acl().put(ACL='public-read') ## sets bucket to public

    ## now let's loop through stim paths and actually upload to s3 (woot!)
    for i,path_to_file in enumerate(full_sketch_paths):
        stim_name = path_to_file.split('/')[-1]
        if ((check_exists(s3, bucket_name, stim_name)==False) | (overwrite==True)):
            print('Now uploading {} | {} of {}'.format(path_to_file.split('/')[-1],(i+1),len(full_sketch_paths)))
            s3.Object(bucket_name,stim_name).put(Body=open(path_to_file,'rb')) ## upload stimuli
            s3.Object(bucket_name,stim_name).Acl().put(ACL='public-read') ## set access controls
        else: 
            print('Skipping {} | {} of {} because it already exists.'.format(path_to_file.split('/')[-1],(i+1),len(full_stim_paths)))
        clear_output(wait=True)

    print('Done uploading images!')

In [ ]:
# example:
# https://iternum.s3.amazonaws.com/iternum-sketches/0074-988d4ee1-5766-47b5-bcbb-49a720aee30d%E2%80%A2bear_5%E2%80%A219%E2%80%A2number%E2%80%A2bear_6_049.png



### Upload catch stims to a different bucket

In [ ]:
catch_bucket_name = 'iternum-recog-catches'
full_catch_paths = h.list_files(path_to_catches)
h.sort_nicely(full_catch_paths)


In [ ]:
reallyRun = 0
if reallyRun: 
    ## tell user some useful information
    print('Path to stimuli is : {}'.format(path_to_catches))
    print('Uploading to this bucket: {}'.format(catch_bucket_name))

    ## establish connection to s3 
#     s3 = boto3.resource('s3')
    s3 = boto3.resource('s3', 
                  region_name='us-east-1', 
                  aws_access_key_id=access_id, #"STORE THIS LOCALLY! Boto3 should be accessing ~/.aws/credentials but it's not for some reason",     
                  aws_secret_access_key= access_key ) #'THIS TOO!'
    
    ## create a bucket with the appropriate bucket name
    try: 
        b = s3.create_bucket(Bucket=catch_bucket_name) 
        print('Created new bucket.')
    except Exception as e:
        print(e)
        b = s3.Bucket(catch_bucket_name)
        print('Bucket already exists.')

    ## do we want to overwrite files on s3?
    overwrite = False
    
    ## set bucket and objects to public
    b.Acl().put(ACL='public-read') ## sets bucket to public

    ## now let's loop through stim paths and actually upload to s3 (woot!)
    for i,path_to_file in enumerate(full_catch_paths):
        stim_name = path_to_file.split('/')[-1]
        if ((check_exists(s3, catch_bucket_name, stim_name)==False) | (overwrite==True)):
            print('Now uploading {} | {} of {}'.format(path_to_file.split('/')[-1],(i+1),len(full_catch_paths)))
            s3.Object(catch_bucket_name,stim_name).put(Body=open(path_to_file,'rb')) ## upload stimuli
            s3.Object(catch_bucket_name,stim_name).Acl().put(ACL='public-read') ## set access controls
        else: 
            print('Skipping {} | {} of {} because it already exists.'.format(path_to_file.split('/')[-1],(i+1),len(full_stim_paths)))
        clear_output(wait=True)

    print('Done uploading images!')